# Intro

[Luminescent AI](https://luminescentai.com/) brings generative AI to photonics and RF. It empowers engineers to create complex optimal components just in a few lines of code! Specifically we created a GPU accelerated and automatic differentiation (AD) compatible FDTD package for simulation and inverse design of photonic integrated circuits (PIC), RF microstrip circuits, antennas and metasurfaces. Experimental release 🥼. Expect bugs 🐛🐞

[Follow us](https://www.linkedin.com/company/luminescent-ai/about) for updates and bug fixes!

# Links
[GitHub](https://github.com/paulxshen/Luminescent.jl): Star us :) We respond to issues within a day  
[LinkedIn](https://www.linkedin.com/company/luminescent-ai/about): Follow us for new features and bug fixes  
[Youtube](https://www.youtube.com/playlist?list=PLaK09N11V08geHeqyLJLDAW0tiDeJtnd4): Video tutorials  
[Company](luminescentai.com): Consulting, collaboration, funding, publication opportunities available  
[White Paper](https://docs.google.com/document/d/1b1kfDdRHw95AsR8zhrZYsHfCHLnULIO7uOYu8PxKQTM/edit?usp=sharing): Generative AI Cells for Semiconductor Photonics Design
Email: pxshen@alumni.stanford.edu info@luminescentai.com  
WhatsApp: 650-776-7724  
WeChat: pxshen1230 懂中文

# Installation
## Windows
## Linux
Please request

# Generative AI inverse design
We introduce GCells (generative cells), a natural evolution of PCells (parametric cells) in semiconductor design . Given a set of inverse design objectives, a GCell will generate optimal geometry using adjoint optimization while ensuring manufacturability by enforcing minimum feature lengths.

In examples below, `gcells.mimo` (multi in multi out) is just a gdsfactory component with configurable waveguide ports, simple slab as pre-optimization geometry, and overlying rectangular design regions. Dimensions `l` along x and `w` along y. Ports are numbered incrementally: west (SW->NW) -> east (SE->NE) -> south (SW->SE) -> north (NW->NE). By default, they're spaced equally on a side. Example: `west=1, east=2` places port 1 on west, ports 2 & 3 on east. But can also individually specify their locations and widths. Example : `west=[1.0, 2.5], wwg_west=[0.5, 0.4]`.

`lmin` is minimum length scale . No fill or void features smaller than `lmin`.

`approx_2D=True` optimizes in 2.5D which is significantly faster than 3D. Examples done at low resolution and lax convergence for speed. For accuracy, the result must be finetuned in 3D at finer resolution, a feature that can be requested from Luminescent AI .


## Generative cells PDK for passive devices 


### 1x2 splitter MMI (tutorial)
1.55um wavelength 1x2 splitter. Symmetric about y so only need to specify T21=1.0. Data saved to `name` folder inside working directory. We start 30 iteratiions of adjoint optimization.

In [1]:
from pprint import pprint
import luminescent as lumi

c = lumi.gcells.mimo(west=1, east=2, l=4.0, w=2.0, wwg=.5, name="1x2_splitter")
targets = {"tparams":{1.55: {"2,1": 0.5}}}

prob = lumi.gcell_problem(
    c, targets, 
    symmetries=[1], lmin=0.15, dx=0.05, 
    approx_2D=True, iters=30, save_memory=True) # set to True if running out of RAM
sol = lumi.solve(prob)
pprint(sol["tparams"])

2024-09-14 19:57:44.738 | WARNING  | kfactory.conf:_validate_layout_cache:254 - 'cell_layout_cache' has been set to True. This might cause when as any cell names generated automatically are loaded from the layout instead of created. This could happen e.g. after reading a gds file into the layout.

          using simulation folder C:\Users\pxshe\OneDrive\Desktop\Luminescent.jl\lumi\1x2_splitter
          started julia process
          compiling julia code...
          
setting up simulation...
using CPU backend.
starting optimization... first iter will be slow due to adjoint compilation.

(1) losses tparams: 0.026863962
weighted total loss 0.026863962
215.256336 seconds (1.04 G allocations: 66.325 GiB, 3.45% gc time, 11.94% compilation time)

(2) losses tparams: 0.49272665
weighted total loss 0.49272665
218.473803 seconds (1.03 G allocations: 65.343 GiB, 5.01% gc time, 0.06% compilation time)

(3) losses tparams: 0.48891023
weighted total loss 0.48891023
236.684172 seconds (1.03 G all

TypeError: load_solution() got an unexpected keyword argument 'path'

Can optimize more. `finetune` acts on latest modified design run folder if `name` not specified. Same for `show_solution` and `load_solution`

In [ ]:
finetune(iters=10)
show_solution()

Can get optimized gdsfactory component, also saved as `optimized_component.gds` in run folder. Can also find `design_region_1.gds` and `design_region_1.png` for just the optimized design regions.

In [ ]:
sol=load_solution()
c_opt=sol["optimized_component"]
c.plot()

### 1x4 splitter MMI
Need same phase on outputs. Because of symmetry, only need 0.0 phase difference between first 2 outputs.

In [ ]:
from pprint import pprint
import luminescent as lumi

c = lumi.gcells.mimo(west=1, east=4, l=4.0, w=4.0, wwg=.5, name="1x4_splitter")
targets = {
    "tparams":{1.55: {"2,1": 0.25, "3,1":0.25}},
    "phasediff":{1.55: {"2,3": 0.0}},
    }

prob = lumi.gcell_problem(
    c, targets, 
    symmetries=[1], lmin=0.1, dx=0.05, 
    approx_2D=True, iters=40, save_memory=True) # set to True if running out of RAM
sol = lumi.solve(prob)

lumi.show_solution()

### wavelength domain demultiplexer

In [ ]:
from pprint import pprint
import luminescent as lumi

c = lumi.gcells.mimo(west=1, east=3, l=6.0, w=6.0, wwg=.5, name="demux")
targets = {"tparams":{
    1.55: {"2,1": 1.0},
    1.31: {"3,1": 1.0},
    1.10: {"4,1": 1.0},
}}

prob = lumi.gcell_problem(
    c, targets,
    lmin=0.1, dx=0.05, 
    approx_2D=True, iters=60, save_memory=True)
sol = lumi.solve(prob)

lumi.show_solution()

### mode converter
"o2@1,o1@0" means converting optical port 1 mode 0 to optical port 2 mode 1.

In [ ]:
from pprint import pprint
import luminescent as lumi

c = lumi.gcells.mimo(west=1, east=1, l=3.0, w=3.0, wwg=.5, name="mode_converter")
targets = {"tparams":{1.55: {"o2@1,o1@0": 1.0}}}

prob = lumi.gcell_problem(
    c, targets,
    lmin=0.1, dx=0.05, 
    approx_2D=True, iters=40, save_memory=True)
sol = lumi.solve(prob)

lumi.show_solution()

### symmetric crossing 

In [ ]:
from pprint import pprint
import luminescent as lumi

c = lumi.gcells.mimo(west=1, east=1, south=1, north=1, l=4.0, w=4.0, wwg=.5, name="crossing")
targets = {"tparams":{1.55: {"2,1": 1.0}}}

prob = lumi.gcell_problem(
    c, targets,
    lmin=0.1, dx=0.05, symmetries=[0,"diag"],
    approx_2D=True, iters=40, save_memory=True)
sol = lumi.solve(prob)

lumi.show_solution()